In [1]:
import numpy as np
import tensorflow as tf

/Users/yunsungsong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sample = ' my name is peter'
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}

In [3]:
input_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sample)-sequence_length):
    x_str = sample[i:i+sequence_length]
    y_str = sample[i+1:i+sequence_length+1]
    x = [char2idx[c] for c in x_str]
    y = [char2idx[c] for c in y_str]
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

In [7]:
X = tf.placeholder(tf.int32, shape=[None,sequence_length])
Y = tf.placeholder(tf.int32, shape=[None,sequence_length])
X_one_hot = tf.one_hot(X, input_dim)

def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)

multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell()]*3, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)
X_for_fc = tf.reshape(outputs, [-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size,sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                                weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
    l, _ = sess.run([loss, train], feed_dict ={X:dataX, Y:dataY})
    if step % 100 == 0:
        print("step:", step, "Loss:", l)

step: 0 Loss: 2.3985415
step: 100 Loss: 0.041660246
step: 200 Loss: 0.04049902
step: 300 Loss: 0.040128354
step: 400 Loss: 0.039956294
step: 500 Loss: 0.039859645
step: 600 Loss: 0.039798535
step: 700 Loss: 0.03975747
step: 800 Loss: 0.039728764
step: 900 Loss: 0.039707854
step: 1000 Loss: 0.039692067
step: 1100 Loss: 0.039679803
step: 1200 Loss: 0.03967006
step: 1300 Loss: 0.039662186
step: 1400 Loss: 0.039655715
step: 1500 Loss: 0.03965033
step: 1600 Loss: 0.039646313
step: 1700 Loss: 0.03964201
step: 1800 Loss: 0.039672296
step: 1900 Loss: 0.039635964


In [23]:
result = sess.run(outputs, feed_dict={X:dataX})
for j, result in enumerate(result):
    index = np.argmax(result, axis=1)
    if j is 0:
        print(''.join([idx2char[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')

my name is peter

In [ ]:
for 